Importamos las librerias

In [18]:
import pandas as pd
import numpy as np
import re
import fun as f

Creamos un data frame de base de datos y observamos la tabla para saber que tipo de hipotesis podemos estudiar.

In [19]:
df = pd.read_csv("../../IH_Shark_Attack_Correlation/input/attacks.csv",encoding='latin')
df.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [20]:
dim= df.shape
f"El documento tiene unas dimensiones de {dim}"

'El documento tiene unas dimensiones de (25723, 24)'

Hipotesis: Los ataques de tiburon hacia deportistas con un equimamiento que les hiciese parecer otro animal no acaban en muerte. Por lo contrario, si el tiburon sabe que ataca a un humano es para matarlo y comer. Evaluaremos tambien que este accidente halla sido o no provocado.

In [21]:
missing_values= (df.isnull().sum()/df.shape[0]).sort_values(ascending=False) .head(10)
missing_values

Unnamed: 22    0.999961
Unnamed: 23    0.999922
Time           0.885394
Species        0.865335
Age            0.865062
Sex            0.776970
Activity       0.776154
Location       0.775998
Fatal (Y/N)    0.775959
Area           0.772694
dtype: float64

Como podemos observar en la anterior celda, tenemos una tabla bastante incompleta para nuestro análisis. Por lo que dirigiremos nuestra hipoteses como he dicho anteriormente, Activity, Fatality y Type.

In [22]:
h_sharks_1 =df[["Activity","Fatal (Y/N)","Type"]]
h_sharks_1.head(30)
h_sharks_1 = h_sharks_1.rename(columns={'Fatal (Y/N)':'Mortalidad',
                                   'Activity':'Deporte','Type':'Tipo'})

Borramos las filas en las que no tengamos informacion en activity, dado que según la actividad que realizaba la victima podremos determinar otras columnas, pero no inversamente.

A continuación hacemos un conteo de las filas con valores nulos para saber con cuantos valores contamos para nuestro valorar nuestra hipotesis.

In [23]:
h_sharks_1.isnull().sum().sort_values(ascending=False)

Deporte       19965
Mortalidad    19960
Tipo          19425
dtype: int64

In [24]:
f.drop_null(h_sharks_1,"Deporte")
f.drop_null(h_sharks_1,"Tipo")
f.drop_null(h_sharks_1,"Mortalidad")
h_sharks_1.isnull().sum().sort_values(ascending=False)

Tipo          0
Mortalidad    0
Deporte       0
dtype: int64

A continuación vamos a observar las diferentes variables que contienen nuestra tabla para tratar de estandarizarlas.

In [25]:
f.c_data(h_sharks_1,"Tipo")

Unprovoked      4259
Provoked         537
Sea Disaster     226
Boating          179
Boat             115
Invalid           22
Questionable       2
Boatomg            1
Name: Tipo, dtype: int64


In [26]:
f.c_data(h_sharks_1,"Mortalidad")


N          4017
Y          1280
UNKNOWN      36
 N            5
M             1
2017          1
y             1
Name: Mortalidad, dtype: int64


In [27]:
f.c_data(h_sharks_1,"Deporte")

Surfing                                        933
Swimming                                       780
Fishing                                        411
Spearfishing                                   309
Bathing                                        149
                                              ... 
The 168-ton Belmore foundered in heavy seas      1
Fishing, attempted to take a netted shark        1
Swept out to sea while fishing                   1
Wreck / Technical diving                         1
Working prawn net                                1
Name: Deporte, Length: 1419, dtype: int64


Como podemos observar, nuestras columnas tienen diferentes datos los cuales vamos a intentar estandarizar. 
1.Tipo: Provoque or Unprovoque. 2. Mortalidad: Yes or No 3. Deporte: Human Shape or Non Human Shape.

Primero empezamos estandarizarnos tipo:

In [28]:
f.standardize_data(h_sharks_1,"Tipo","Sea Disaster","Unprovoked")
f.standardize_data(h_sharks_1,"Tipo","Boat","Unprovoked")
f.standardize_data(h_sharks_1,"Tipo","Boating","Unprovoked")
f.standardize_data(h_sharks_1,"Tipo","Invalid","Unprovoked")
f.standardize_data(h_sharks_1,"Tipo","Questionable","Unprovoked")
f.standardize_data(h_sharks_1,"Tipo","Boatomg","Unprovoked")

In [29]:
f.c_data(h_sharks_1,"Tipo")
drop_unresponsive= ['Unprovoked']
h_sharks_1= h_sharks_1[(h_sharks_1["Tipo"].isin(drop_unresponsive))]
f.c_data(h_sharks_1,"Tipo")

Unprovoked    4804
Provoked       537
Name: Tipo, dtype: int64
Unprovoked    4804
Name: Tipo, dtype: int64


A continuación eliminamos las filas de Mortalidad desconocida.

In [30]:
drop_mortal= ['UNKNOWN', ' N','M','2017','y','N ']
h_sharks_1= h_sharks_1[~(h_sharks_1["Mortalidad"].isin(drop_mortal))]
f.c_data(h_sharks_1,"Mortalidad")

N    3503
Y    1263
Name: Mortalidad, dtype: int64


Por último limpiamos deporte haciendo uso de regex.

Para ello vamos a analizar las variables mas repetidas en nuestra tabla.


In [31]:
dim1=h_sharks_1.shape
(h_sharks_1["Deporte"].value_counts()/dim1[0]).head(30)

Surfing                0.192195
Swimming               0.159883
Spearfishing           0.053714
Fishing                0.049727
Bathing                0.030634
Wading                 0.027696
Diving                 0.020562
Standing               0.020143
Snorkeling             0.017205
Body boarding          0.012799
Scuba diving           0.011960
Body surfing           0.009652
Swimming               0.008603
Pearl diving           0.006504
Kayaking               0.006504
Treading water         0.006295
Fell overboard         0.006085
Boogie boarding        0.005245
Free diving            0.004616
Windsurfing            0.003987
Walking                0.003147
Boogie Boarding        0.003147
Floating               0.002728
Surf skiing            0.002518
Rowing                 0.002308
Sponge diving          0.002098
Canoeing               0.002098
Surf-skiing            0.002098
Fell into the water    0.001888
Diving for trochus     0.001888
Name: Deporte, dtype: float64

In [32]:
activity_rename={
".*[Ss]urfing":"Not Human Shape",
".*[Ss]wimming":"Human Shape",
".*[Ff]ishing":"Human Shape",
".*[Ss]pearfishing":"Human Shape",
".*[Bb]athing":"Human Shape",
".*[Ww]ading":"Human Shape",
".*[Dd]iving":"Human Shape",
".*[Ss]tanding":"Human Shape",
".*[Ss]norkeling":"Human Shape",
".*[Ss]cuba":"Human Shape",
".*[Bb]ody":"Not Human Shape",
".*[Bb]ody":"Not Human Shape",
".*[Ss]wimming":"Human Shape",
".*[Pp]earl":"Human Shape",
".*[Kk]ayaking":"Not Human Shape",
".*[Tt]reading":"Not Human Shape",
".*[Ff]ell":"Human Shape",
".*[Ff]ree":"Human Shape",
".*[Bb]oogie":"Not Human Shape",
".*[Ww]indsurfing":"Not Human Shape",
".*[Bb]oogie":"Not Human Shape",
".*[Ww]alking":"Human Shape",
".*[Ff]loating":"Human Shape",
".*[Ff]ishing":"Human Shape",
".*[Rr]owing":"Not Human Shape",
".*[Cc]anoeing":"Not Human Shape",
".*[Ss]urf":"Not Human Shape",
".*[Kk]ayak":"Not Human Shape"}

In [33]:
Shapes =[]
for i in h_sharks_1.Deporte:
    for key,value in activity_rename.items():
        if re.search(key, str(i)):
            i=value
    Shapes.append(i) 

In [34]:
h_sharks_1.Deporte= Shapes

In [35]:
dim1=h_sharks_1.shape
(h_sharks_1["Deporte"].value_counts()/dim1[0]).head(5)

Human Shape        0.595468
Not Human Shape    0.282417
Paddle boarding    0.001679
Playing            0.001469
Sailing            0.001469
Name: Deporte, dtype: float64

Como podemos observar en la celdas de arriba, hemos conseguido estandarizar los datos que teniamos en un 86%, procedemos a eliminar el resto de filas.

In [36]:
drop_mortal= ['Human Shape','Not Human Shape']
h_sharks_1= h_sharks_1[(h_sharks_1["Deporte"].isin(drop_mortal))]
f.c_data(h_sharks_1,"Deporte")

Human Shape        2838
Not Human Shape    1346
Name: Deporte, dtype: int64


In [37]:
h_sharks_1 = h_sharks_1.reset_index(drop=True)

In [38]:
mortality_by_shape = h_sharks_1.groupby(["Deporte","Mortalidad"]).agg({"Mortalidad":"count"})



In [39]:
mortality_by_shape

Mortalidad
Deporte         Mortalidad            
Human Shape     N                 1954
                Y                  884
Not Human Shape N                 1244
                Y                  102

Conclusión: Podemos verificar que la hipótesis se cumple. Un 28% de los atacados con Human Shape, fallecieron frente a un 8% en caso de que estuviesen haciendo deporte.  